In [35]:
import jieba
jieba_dict_path = 'data/jieba/dict.txt.big'
jieba_userdict_path = 'data/jieba/userdict.datetime.txt'
jieba_stopwords_path = 'data/jieba/stop_words.txt'

jieba.set_dictionary(jieba_dict_path) # Set Chinese Traditional
jieba.load_userdict(jieba_userdict_path) # Load User Customized Dict
# with open(r'%s' % self.jieba_stopwords_path, 'r', encoding='utf8') as f:
#     self.jieba_stopword_list = [line.strip() for line in f]
jieba.initialize() # Init at startup

Building prefix dict from /Users/ryan/Python/github/Python-common-function/data/jieba/dict.txt.big ...
Loading model from cache /var/folders/lm/565c_3js07301sjgqpyjwpy00000gn/T/jieba.u7eb4bf76bab733c7ca02ccc176993051.cache
Loading model cost 1.264 seconds.
Prefix dict has been built succesfully.


In [75]:
#日期识别



import re
from datetime import datetime,timedelta
from dateutil.parser import parse
import jieba.posseg as psg

UTIL_CN_NUM = {
    '零': 0, '一': 1, '二': 2, '兩': 2, '三': 3, '四': 4,
    '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
    '5': 5, '6': 6, '7': 7, '8': 8, '9': 9
}

UTIL_CN_UNIT = {'十': 10, '百': 100, '千': 1000, '萬': 10000}

def cn2dig(src):
    if src == "":
        return None
    m = re.match("\d+", src)
    if m:
        return int(m.group(0))
    rsl = 0
    unit = 1
    for item in src[::-1]:
        if item in UTIL_CN_UNIT.keys():
            unit = UTIL_CN_UNIT[item]
        elif item in UTIL_CN_NUM.keys():
            num = UTIL_CN_NUM[item]
            rsl += num * unit
        else:
            return None
    if rsl < unit:
        rsl += unit
    return rsl

def year2dig(year):
    res = ''
    for item in year:
        if item in UTIL_CN_NUM.keys():
            res = res + str(UTIL_CN_NUM[item])
        else:
            res = res + item
    m = re.match("\d+", res)
    if m:
        if len(m.group(0)) == 2:
            return int(datetime.datetime.today().year/100)*100 + int(m.group(0))
        else:
            return int(m.group(0))
    else:
        return None

def parse_datetime(msg):
    if msg is None or len(msg) == 0:
        return None

    print('parse_datetime, msg: %s' % msg)
    try:
        dt = parse(msg, fuzzy=True)
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except Exception as e:
        print('parse_datetime try to change chinese char')
        m = re.match(
            r"([0-9零一二两兩三四五六七八九十]+年)?([0-9一二两兩三四五六七八九十]+月)?([0-9一二两兩三四五六七八九十]+[号號日])?([上中下午晚早]+)?([0-9零一二两三四五六七八九十百]+[点:\.时])?([0-9零一二三四五六七八九十百]+分?)?([0-9零一二三四五六七八九十百]+秒)?",
            msg)
        if m.group(0) is not None:
            res = {
                "year": m.group(1),
                "month": m.group(2),
                "day": m.group(3),
                "hour": m.group(5) if m.group(5) is not None else '00',
                "minute": m.group(6) if m.group(6) is not None else '00',
                "second": m.group(7) if m.group(7) is not None else '00',
            }
            params = {}
            print('res: %s' %s)
            
            for name in res:
                if res[name] is not None and len(res[name]) != 0:
                    tmp = None
                    if name == 'year':
                        tmp = year2dig(res[name][:-1])
                    else:
                        tmp = cn2dig(res[name][:-1])
                    if tmp is not None:
                        params[name] = int(tmp)
            target_date = datetime.today().replace(**params)
            is_pm = m.group(4)
            if is_pm is not None:
                if is_pm == u'下午' or is_pm == u'晚上' or is_pm =='中午':
                    hour = target_date.time().hour
                    if hour < 12:
                        target_date = target_date.replace(hour=hour + 12)
            return target_date.strftime('%Y-%m-%d %H:%M:%S')
        else:
            return None



def check_time_valid(word):
    m = re.match("\d+$", word)
    if m:
        if len(word) <= 6:
            return None
    word1 = re.sub('[号|号|號|日]\d+$', '日', word)
    if word1 != word:
        return check_time_valid(word1)
    else:
        return word1

#时间提取
def time_extract(text):

    time_res = []
    word = ''
    keyDate = {'今天': 0, '明天':1, '聽日': 1, '后天': 2, '後天': 2, '後日': 2,}
    for k, v in psg.cut(text):
        if k in keyDate:
            if word != '':
                time_res.append(word)
            word = (datetime.today() + timedelta(days=keyDate.get(k, 0))).strftime('%Y年%m月%d日')
            print('word: %s' % word)
        elif word != '':
            if v in ['m', 't']:
                word = word + k
            else:
                time_res.append(word)
                word = ''
        elif v in ['m', 't']:
            word = k
    if word != '':
        time_res.append(word)
    print('time_res: %s' % time_res)
    result = list(filter(lambda x: x is not None, [check_time_valid(w) for w in time_res]))
    print('result: %s' % result)
    final_res = [parse_datetime(w) for w in result]

    return [x for x in final_res if x is not None]



In [76]:
text1 = '我要住到明天下午3點'
for v, k in psg.cut(text1):
    print(v, k)
time_extract(text1)

我 r
要 v
住 v
到 v
明天 t
下午 t
3 m
點 m
word: 2019年03月04日
time_res: ['2019年03月04日下午3點']
result: ['2019年03月04日下午3點']
parse_datetime, msg: 2019年03月04日下午3點


['2003-03-04 00:00:00']

In [72]:
t = '2019年03月04日下午3點'
t.strftime()

AttributeError: 'str' object has no attribute 'strftime'

In [52]:
text1 = '我要住到明天下午三点'
print(text1, time_extract(text1), sep=':')

text2 = '预定28号4点的房间'
print(text2, time_extract(text2), sep=':')

text3 = '我要从26号下午4点住到11月2号'
print(text3, time_extract(text3), sep=':')

text4 = '我要预订今天到30號的房间'
print(text4, time_extract(text4), sep=':')

text5 = '今天30号呵呵'
print(text5, time_extract(text5), sep=':')

word: 2019年03月04日
我要住到明天下午三点:['2019-03-04 00:00:00']
预定28号4点的房间:['2019-03-28 04:00:00']
我要从26号下午4点住到11月2号:['2019-03-26 16:00:00', '2019-11-02 00:00:00']
word: 2019年03月03日
我要预订今天到30號的房间:['2019-03-03 00:00:00', '2019-03-30 00:00:00']
word: 2019年03月03日
今天30号呵呵:['2030-03-03 00:00:00']
